In [ ]:
# import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import colors
from sklearn.preprocessing import LabelEncoder
from category_encoders import OrdinalEncoder

In [ ]:
# load file
data = pd.read_csv('shopping_behavior_updated.csv')
print("Number of datapoints:", len(data))
data.head()

In [ ]:
# data cleaning
data.info()

There is no missing value in the dataset.

In [ ]:
# data exploration

# Get list of categorical variables
objs = (data.dtypes == 'object')
object_cols = list(objs[objs].index)
print("Categorical variables:")
print(object_cols)


In [ ]:
# Get a look
print(data['Gender'].value_counts())
print('----------------------------------')
print(data['Item Purchased'].value_counts())
print('----------------------------------')
print(data['Category'].value_counts())
print('----------------------------------')
print(data['Location'].value_counts())
print('----------------------------------')
print(data['Size'].value_counts())
print('----------------------------------')
print(data['Color'].value_counts())
print('----------------------------------')
print(data['Season'].value_counts())
print('----------------------------------')
print(data['Subscription Status'].value_counts())
print('----------------------------------')
print(data['Shipping Type'].value_counts())
print('----------------------------------')
print(data['Discount Applied'].value_counts())
print('----------------------------------')
print(data['Promo Code Used'].value_counts())
print('----------------------------------')
print(data['Payment Method'].value_counts())
print('----------------------------------')
print(data['Frequency of Purchases'].value_counts())

In [ ]:
le=LabelEncoder()
data['Location'] = le.fit_transform(data['Location'])
data['Category'] = le.fit_transform(data['Category'])
#To plot some selected features 
#Setting up colors prefrences
sns.set(rc={"axes.facecolor":"#FFF9ED","figure.facecolor":"#FFF9ED"})
pallet = ["#682F2F", "#9E726F", "#D6B2B1", "#B9C0C9", "#9F8A78", "#F3AB60"]
cmap = colors.ListedColormap(["#682F2F", "#9E726F", "#D6B2B1", "#B9C0C9", "#9F8A78", "#F3AB60"])
#Plotting following features
To_Plot = [ "Location", "Category"]
print("Reletive Plot Of Some Selected Features: A Data Subset")
plt.figure()
sns.pairplot(data[To_Plot])
#Taking hue 
plt.show()

Promo Code Used is the same as discount applied. So we can drop one of them.\
Features with level:'Subscription Status','Shipping Type', 'Discount Applied','Payment Method', 'Frequency of Purchases'\
Features 'Location' does not have noteworthy correlation with goods,and its one-hot coding will be large,so drop it.\

In [ ]:
# Drop some columns
to_drop = ['Promo Code Used', 'Customer ID', 'Location']
data.drop(to_drop, axis=1, inplace=True)
object_cols.remove('Promo Code Used')
object_cols.remove('Location')

In [ ]:
# encoding

# with level,ordinal encoding
ord_map1 = {'No':0, 'Yes':1}
ord_map2 = {'Store Pickup':0, 'Free Shipping':0, 'Standard':1,'2-Day Shipping':2,'Express':3,'Next Day Air':4}
ord_map3 = {'No':1, 'Yes':0}
ord_map4 = {'Debit Card':0,'Credit Card':0, 'PayPal':1, 'Bank Transfer':2, 'Cash':2,'Venmo':2}
ord_map5 = {'Annually':1, 'Quarterly':2,'Every 3 Month':3,'Monthly':4,'Fortnightly':5,'Bi-Weekly':5,'Weekly':6}
ord_map6 = {'S':1,'M':2,'L':3,'XL':4}

data['Subscription Status'] = data['Subscription Status'].map(ord_map1)
data['Shipping Type'] = data['Shipping Type'].map(ord_map2)
data['Discount Applied'] = data['Discount Applied'].map(ord_map3)
data['Payment Method'] = data['Payment Method'].map(ord_map4)
data['Frequency of Purchases'] = data['Frequency of Purchases'].map(ord_map5)
data['Size'] = data['Size'].map(ord_map6)
data.head()

In [ ]:
# without level, one-hot encoding
print(object_cols)
object_cols.remove('Subscription Status')
object_cols.remove('Shipping Type')
object_cols.remove('Discount Applied')
object_cols.remove('Payment Method')
object_cols.remove('Frequency of Purchases')
object_cols.remove('Size')
print(object_cols)
for i in object_cols:
    data = pd.concat([data,pd.get_dummies(data[i],prefix=i,dtype=int)],axis=1)
    data.drop(i,axis=1,inplace=True)

data.head()